#### libics: drivers

# Vialux V-7000 / Texas Instruments DLP7000

Product homepage: [Vialux V-7000](https://www.vialux.de/de/hi-speed-v-module.html)

### Imports

In [ ]:
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from libics import env
from libics.drv import drv
from libics.drv.itf import itf
from libics.data import types, dataset, stp
from libics.display import plot, plotdefault
from libics.file import hdf

### Configuration

In [ ]:
itf_cfg = {
    "protocol": itf.ITF_PROTOCOL.BINARY,
    "interface": itf.ITF_BIN.VIALUX,
    "device": None
}
itf_cfg = itf.ProtocolCfgBase(**itf_cfg).get_hl_cfg()

In [ ]:
drv_cfg = {
    "driver": drv.DRV_DRIVER.DSP,
    "interface": itf_cfg,
    "identifier": "dmd_vialux_v7000_texasinstruments_dlp7000",
    "model": drv.DRV_MODEL.TEXASINSTRUMENTS_DLP7000,
    "pixel_hrzt_count": 1024,
    "pixel_hrzt_size": 13.68e-6,
    "pixel_hrzt_offset": 0,
    "pixel_vert_count": 768,
    "pixel_vert_size": 13.68e-6,
    "pixel_vert_offset": 0,
    "format_color": drv.DRV_DSP.FORMAT_COLOR.BW,
    "channel_bitdepth": 1,
    "picture_time": 9,
    "dark_time": 0,
    "sequence_repetitions": 0,
    "temperature": 25.0
}
drv_cfg = drv.DrvCfgBase(**drv_cfg).get_hl_cfg()

### Setup

In [ ]:
dsp = drv.DrvBase(cfg=drv_cfg).get_drv()
sleep_time = 5.0

### Script

##### Create data

In [ ]:
val = 96
im1 = np.arange(1024 * 768, dtype=float).reshape((1024, 768))
im1[im1 % val < val / 2] = 0
im1[im1 % val >= val / 2] = 1
plt.subplot(121)
plt.imshow(im1.T)
im2 = np.full((1024, 768), 0, dtype=float)
plt.subplot(122)
plt.imshow(im2.T)
im = [im1, im2]

##### Set image

In [ ]:
with dsp:
    dsp.init(im)
    dsp.run()
    time.sleep(sleep_time)
    dsp.stop()